# Bài 1: phân chia loài hoa Iris

# Import thư viện

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report


# Load dữ liệu

In [12]:
df = pd.read_csv('./data/iris.data', header=None)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

df.head()

0    1    2    3            4
0  5.1  3.5  1.4  0.2  Iris-setosa
1  4.9  3.0  1.4  0.2  Iris-setosa
2  4.7  3.2  1.3  0.2  Iris-setosa
3  4.6  3.1  1.5  0.2  Iris-setosa
4  5.0  3.6  1.4  0.2  Iris-setosa

# Thông tin data

In [13]:
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       150 non-null    float64
 1   1       150 non-null    float64
 2   2       150 non-null    float64
 3   3       150 non-null    float64
 4   4       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


0    0
1    0
2    0
3    0
4    0
dtype: int64

# Chuẩn bị dữ liệu

In [14]:
# Chia dữ liệu thành train/test theo tỷ lệ 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# Cài đặt Gaussian Naive Bayes thủ công


In [15]:
class MyGaussianNB:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = {}
        self.var = {}
        self.priors = {}

        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = X_c.mean(axis=0)
            self.var[c] = X_c.var(axis=0)
            self.priors[c] = X_c.shape[0] / X.shape[0]

    def gaussian_pdf(self, x, mean, var):
        eps = 1e-6
        return (1 / np.sqrt(2 * np.pi * (var + eps))) * np.exp(- (x - mean)**2 / (2 * (var + eps)))

    def predict_one(self, x):
        posteriors = {}

        for c in self.classes:
            prior = np.log(self.priors[c])
            likelihood = np.sum(np.log(self.gaussian_pdf(x, self.mean[c], self.var[c])))
            posteriors[c] = prior + likelihood
        return max(posteriors, key=posteriors.get)

    def predict(self, X):
        return np.array([self.predict_one(x) for x in X])


# Train Naive Bayes thủ công


In [16]:
nb = MyGaussianNB()
nb.fit(X_train, y_train)

y_pred_nb = nb.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))


Accuracy:  1.0
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30



# Huấn luyện mô hình Naive Bayes

In [6]:
model = GaussianNB()
model.fit(X_train, y_train)


GaussianNB()

# Dự đoán

In [7]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30



# Tóm lại
+ Accuracy khoảng 0.93 – 1.0 tùy train/test split
+ chạy nhanh không cần tuning nhiều
+ hoạt động tốt với dữ liệu nhỏ